In [1]:
import os


os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

device = 'cuda'

In [2]:
import argparse
import itertools
import os
import time

import torch
import torch._inductor.config
from fms.models import get_model
from fms.utils import generation, tokenizers
from torch import distributed as dist

import fms_extras.models.paged_gpt_bigcode
import fms_extras.models.paged_llama
from fms_extras.models.speculator import MLPSpeculator
from fms_extras.utils.generation import paged_generate, speculative_generate


/root/work/fms-extras/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from fms.models import get_model


get_model

<function fms.models.get_model(architecture: str, variant: str, model_path: Optional[str] = None, source: Optional[str] = None, device_type: str = 'cpu', distributed_strategy: Optional[str] = None, checkpoint_sharding: Optional[str] = None, group: Optional[torch.distributed.distributed_c10d.ProcessGroup] = None, **kwargs)>

In [4]:
import torch
#from torch import distributed as dist

import fms_extras.models.paged_gpt_bigcode
import fms_extras.models.paged_llama


torch.set_default_dtype(torch.half)

model = get_model(
    f"paged_llama",
    'llama3.8b',
    model_path='/models/Meta-Llama-3-8B-Instruct',
    checkpoint_sharding=None,
    device_type=device,
    source='hf',
    distributed_strategy=None,
    group=None,
)

model

architecture='paged_llama', variant='llama3.8b', model_path='/models/Meta-Llama-3-8B-Instruct', source='hf', device_type='cuda', distributed_strategy=None, checkpoint_sharding=None, group=None, kwargs={}


PagedLLaMA(
  (headless_model): PagedLLaMAHeadless(
    (shared): WordEmbedding(
      (emb): Embedding(128256, 4096)
      (head): Linear(in_features=4096, out_features=128256, bias=False)
    )
    (layers): ModuleList(
      (0-31): 32 x PagedLLaMABlock(
        (ln): LayerNormParameterized()
        (ff_ln): LayerNormParameterized()
        (attn): PagedMultiHeadAttention(
          (dense): Linear(in_features=4096, out_features=4096, bias=False)
          (qkv_fused): Linear(in_features=4096, out_features=6144, bias=False)
        )
        (ff_sub_layer): GatedLinearUnit(
          (wg1_fused): Linear(in_features=4096, out_features=28672, bias=False)
          (a): SiLU()
          (w2): Linear(in_features=14336, out_features=4096, bias=False)
        )
      )
    )
    (dec_norm): LayerNormParameterized()
  )
  (head): WordEmbedding(
    (emb): Embedding(128256, 4096)
    (head): Linear(in_features=4096, out_features=128256, bias=False)
  )
)

In [5]:
import torch
from fms.utils import generation, tokenizers


tokenizer = tokenizers.get_tokenizer('/models/Meta-Llama-3-8B-Instruct')
model.eval()
torch.set_grad_enabled(False)


In [6]:
from fms_extras.models.hf.modeling_mlp_speculator import MLPSpeculatorPreTrainedModel


speculator = MLPSpeculatorPreTrainedModel.from_pretrained(
    'ibm-fms/llama3-8b-accelerator', device_map=device
).speculator
speculator

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


MLPSpeculator(
  (emb): ModuleList(
    (0-3): 4 x Embedding(128256, 3072)
  )
  (proj): ModuleList(
    (0): Linear(in_features=4096, out_features=3072, bias=False)
    (1-3): 3 x Linear(in_features=3072, out_features=3072, bias=False)
  )
  (head): ModuleList(
    (0-3): 4 x Linear(in_features=3072, out_features=128256, bias=False)
  )
  (ln): ModuleList(
    (0-3): 4 x LayerNormParameterized()
  )
  (activation): GELU(approximate='none')
)

In [7]:
speculator.n_predict

4

In [8]:
model.config.nheads, model.config.kvheads

(32, 8)

In [9]:
from fms_extras.utils.cache.paged import PagedKVCacheManager


kv_cache_manager = PagedKVCacheManager(
    model.config.nlayers,
    model.config.nheads,
    model.config.emb_dim,
    kv_heads=model.config.kvheads,
    tensor_parallel_size=1,
    dtype=torch.get_default_dtype(),
    device=device,
)
kv_cache_manager

total_gpu_memory=25386352640, gpu_memory_utilization=0.96, peak_memory=22458458112, cache_block_size=8388608


In [10]:
tokenizer.bos_token_id != tokenizer.eos_token_id

True

In [11]:
template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{}\n\n### Response:"

prompt1 = template.format(
    "Provide a list of instructions for preparing chicken soup."
)
prompt1

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nProvide a list of instructions for preparing chicken soup.\n\n### Response:'

In [12]:
def ids_for_prompt(prompt):
    tokens = tokenizer.tokenize(prompt)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    ids = [tokenizer.bos_token_id] + ids
    ids = torch.tensor(ids, dtype=torch.long, device=device)
    return ids

prompt1 = ids_for_prompt(prompt1)
prompt1

tensor([128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
            13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
           382,  14711,  30151,    512,  61524,    264,   1160,    315,  11470,
           369,  20646,  16553,  19724,    382,  14711,   6075,     25],
       device='cuda:0')

In [13]:
ids = [prompt1]

In [14]:
cudagraphs = True
max_seq_len = (
    model.config.max_expected_seq_len
    if hasattr(model.config, "max_expected_seq_len")
    else model.config.max_pos
)
max_seq_len


8192

In [ ]:
#result, n_steps, ttft, generated_token_time_out = paged_generate(
#    model,
#    ids,
#    kv_cache_manager,
#    max_new_tokens=100,
#    max_seq_len=max_seq_len,
#    do_sample=False,
#    decode_model=None,
#    cudagraphs=cudagraphs,
#)


In [16]:
from fms_extras.utils.generation import paged_generate, speculative_generate


result, n_steps, ttft, generated_token_time_out, n_accepts = speculative_generate(
    model,
    ids,
    speculator,
    kv_cache_manager,
    new_tokens=100,
    max_seq_len=max_seq_len,
    decode_model=None,
    # todo: we can only reduce-overhead for now when batch size is 1
    flattening=True,
    cudagraphs=False,
    threshes=[4,3,2,2],
)

result, n_steps, ttft, generated_token_time_out, n_accepts


([tensor([128000,  39314,    374,    459,   7754,    430,  16964,    264,   3465,
              13,   9842,    264,   2077,    430,  36001,  45695,    279,   1715,
             382,  14711,  30151,    512,  61524,    264,   1160,    315,  11470,
             369,  20646,  16553,  19724,    382,  14711,   6075,     25,   4815,
            8586,    374,    264,   1160,    315,  11470,    369,  20646,  16553,
           19724,   1473,    334,   8468,    220,     16,     25,  50095,  52275,
           57277,      9,    220,     16,  31123,  17685,   1752,     11,   6930,
            1752,  16553,  17659,    477,  60611,    198,      9,    220,     19,
           26446,  16553,  45993,    198,      9,    220,     16,   3544,  38427,
              11,  38525,    198,      9,    220,     18,  85388,  31735,     11,
           94927,    198,      9,    220,     17,  62517,     11,  83612,    323,
           38525,    198,      9,    220,     17,  70121,  55972,     82,     11,
           38525